<a href="https://colab.research.google.com/github/RachelRamirez/misclassification_matrix/blob/main/Potential_Misclassification_Cost_Matrix_Example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Now that I've shown the TB and ISantaro methods are equivalent when seeded properly, I believe what I need to do is train a "good" neural network with the same seed 30 times to get an idea of what the typical confusion matrix looks like.  I will run the code within Colab 30 times without restarting because from previous reproducibility studies if I restart and run-all I will get the same results.  This time, I'm interested in the typical variety you can get on a confusion matrix when the neural network is seeded the same way each time, that way when I gve it another cost-matrix to train on, and run that 30 times I can do a more informative comparison of the results.  Since the Isantaro and TB methods were identical I went with the Isantaro method because it was simpler, more efficient, and seemed less time consuming. 

Changes from previous Reproducibility notebook:
1. Dropout is back in.
2. Batch Size is not as large to help with variety.
3. Num of Epochs is more than 4 now that I care about achieving good overall accuracy
4. Callback for EarlyStop added
5. Model Shuffle during Fit is still False (I'm calling it out to see if I need to change that)
6. but Model.Fit(use multiprocessors = True)
7. Still Cost Matrix of all 1's

### Reproducible Seeds

In [1]:
#For Reproducibility
import numpy as np
# np.random.seed(1337)  # for reproducibility

import tensorflow as tf
# tf.random.set_seed(33)

import random as python_random
# python_random.seed(4)

# https://www.tensorflow.org/api_docs/python/tf/keras/utils/set_random_seed
tf.keras.utils.set_random_seed(342) #Possibly use next iteration if the above doesn't work


# Running more than once causes variation.  try adding this:
# Set seed value
seed_value = 56
import os
os.environ['PYTHONHASHSEED']=str(seed_value)

print("TF version: " , tf.__version__ )
print("Keras version: " , tf.keras.__version__ )

TF version:  2.9.2
Keras version:  2.9.0


### Import rest of Library

In [2]:

# from https://github.com/keras-team/keras/issues/2115#issuecomment-204060456
# witha correction on the weighted function in the middle 

'''Train a simple deep NN on the MNIST dataset.
Get to 98.40% test accuracy after 20 epochs
(there is *a lot* of margin for parameter tuning).
2 seconds per epoch on a K520 GPU.
'''

from __future__ import print_function  #do i still need this?
from keras.datasets import mnist
from keras.models import Sequential
from keras.layers.core import Dense, Dropout, Activation
from keras.optimizers import SGD, Adam, RMSprop
from keras.utils import np_utils
import keras.backend as K
from itertools import product
import functools
from functools import partial

from time import sleep
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd 
from sklearn.metrics import confusion_matrix

## MORE REPEATABILITY STUFF NEEDED - If theres a way to update this to V2 of Tensorflow great, otherwise I had to use TF 1.0 code
# 5. Configure a new global `tensorflow` session (https://stackoverflow.com/questions/50659482/why-cant-i-get-reproducible-results-in-keras-even-though-i-set-the-random-seeds)
# from keras import backend as K


#I believe thecode below is to help things be repeatable each time different sections in my google colab notebook execute
session_conf = tf.compat.v1.ConfigProto(intra_op_parallelism_threads=1, inter_op_parallelism_threads=1)
sess = tf.compat.v1.Session(graph=tf.compat.v1.get_default_graph(), config=session_conf)
K.set_session(sess)

### Define batch, epochs, and format data

In [3]:
batch_size = 256 # I originally had it very  high batch size to reduce the variation in the data each batch and hope it makes the model training more nearly identical which it did, then i bring it back down to something reasonable to get better results training the NN
nb_classes = 10
nb_epoch = 15

# the data, shuffled and split between train and test sets
(X_train, y_train), (X_test, y_test) = mnist.load_data()


X_train = X_train.reshape(60000, 784)
X_test = X_test.reshape(10000, 784)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255
print(X_train.shape[0], 'train samples')
print(X_test.shape[0], 'test samples')

# convert class vectors to binary class matrices
Y_train = np_utils.to_categorical(y_train, nb_classes)
Y_test = np_utils.to_categorical(y_test, nb_classes)


11490434/11490434 [==============================] - 2s 0us/step
60000 train samples
10000 test samples


### Define weighted_categorical_crossentropy()

In [4]:

# # https://github.com/keras-team/keras/issues/2115#issuecomment-207765342

# def w_categorical_crossentropy(y_true, y_pred, weights):
#     nb_cl = len(weights)
#     final_mask = K.zeros_like(y_pred[:, 0])
#     y_pred_max = K.max(y_pred, axis=1)
#     y_pred_max = K.expand_dims(y_pred_max, 1)
#     y_pred_max_mat = K.equal(y_pred, y_pred_max)
 
#     for c_t, c_p in product(range(nb_cl), range(nb_cl)):
#         final_mask += (K.cast(weights[c_t, c_p],K.floatx()) * K.cast(y_pred_max_mat[:, c_p] ,K.floatx())* K.cast(y_true[:, c_t],K.floatx()))
    
#     # result = K.categorical_crossentropy(y_true, y_pred)*final_mask
#     # tf.print(result, "Show Result of CE * Final_Mask")  #this was basically useless to display, and it showed like, 500 lines of print statements each epoch

#     return K.categorical_crossentropy(y_true, y_pred)*final_mask   #I changed the order of y_true and y_pred




### Define Same Model but use normal Categorical CrossEntropy with no extra cost-matrix of Weights

In [6]:
def normal_method():

  model = Sequential()
  model.add(Dense(512, input_shape=(784,) ,kernel_initializer=tf.keras.initializers.glorot_uniform(seed=42)))
  model.add(Activation('relu'))
  model.add(Dropout(0.2))
  model.add(Dense(512, kernel_initializer=tf.keras.initializers.glorot_uniform(seed=42)))
  model.add(Activation('relu'))
  model.add(Dropout(0.2))
  model.add(Dense(10, kernel_initializer=tf.keras.initializers.glorot_uniform(seed=42)))
  model.add(Activation('softmax'))

  rms = RMSprop()
  # model.compile(loss=ncce, optimizer=rms)
  model.compile(loss=tf.keras.losses.CategoricalCrossentropy(), optimizer=rms, metrics='categorical_accuracy', )

  #add early_stop to prevent overfittings
  # callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

  model_history = model.fit(X_train, Y_train,
            batch_size=batch_size, epochs=nb_epoch, verbose=2,
            validation_data=(X_test, Y_test),shuffle=False, use_multiprocessing=True
            , callbacks = [callback])

  
  # model.evaluate(X_test, Y_test, verbose=1)  # I know this isn't the typical use of train/val/test sets, please dont' comment on that
  
  #Predict
  y_prediction = model.predict(X_test)
  y_prediction  = np.argmax(y_prediction, axis=1)
  # Y_prediction = np_utils.to_categorical(y_prediction, nb_classes) #If I want to do SparseCategoricalAccuracy

  #Create confusion matrix and normalizes it over predicted (columns)
  # result = confusion_matrix(y_test, y_prediction , normalize='pred')  #if I want percentages instead of raw counts

  
  cm = confusion_matrix(y_test, y_prediction)
  cm = pd.DataFrame(cm, range(10),range(10))

  #This shows a pretty confusion matrix which I don't neeed to show right now
  # plt.figure(figsize = (10,10))
  # sns.heatmap(cm, annot=True, annot_kws={"size": 12}) # font size
  # plt.show()
  # cm_normal = cm

  return cm


### Weighted Categorical Cross Entropy Function

In [7]:
# from https://github.com/keras-team/keras/issues/2115#issuecomment-815825633 from Isaranto

def weighted_categorical_crossentropy_new(y_true, y_pred, weights):
          idx1 = K.argmax(y_pred, axis=1)
          idx2 = K.argmax(y_true, axis=1)
          mask = tf.gather_nd(weights, tf.stack((idx1, idx2), -1))
          return K.categorical_crossentropy(y_true, y_pred) * mask

In [8]:
 #what does ncce stand for?

 
def isaranto_method():
  w_array = np.ones((10,10))
  # w_array[9, 7] = 1.5
  # w_array = w_array - np.eye(10)
  # print("W_array:  ", w_array)

  weighted_list = w_array.tolist()

  wcce = partial(weighted_categorical_crossentropy_new, weights=weighted_list)
  wcce.__name__ ='w_categorical_crossentropy'

  model3 = Sequential()
  model3.add(Dense(512, input_shape=(784,), kernel_initializer=tf.keras.initializers.glorot_uniform(seed=42)))
  model3.add(Activation('relu'))
  model3.add(Dropout(0.2))
  model3.add(Dense(512, kernel_initializer=tf.keras.initializers.glorot_uniform(seed=42)))
  model3.add(Activation('relu'))
  model3.add(Dropout(0.2))
  model3.add(Dense(10,kernel_initializer=tf.keras.initializers.glorot_uniform(seed=42)))
  model3.add(Activation('softmax'))

  rms = RMSprop()

  model3.compile(loss=wcce, optimizer=rms,  metrics='categorical_accuracy',)
  callback = tf.keras.callbacks.EarlyStopping(monitor='val_loss', patience=3)

  model3_history = model3.fit(X_train, Y_train,
            batch_size=batch_size, epochs=nb_epoch, verbose=2,
            validation_data=(X_test, Y_test), shuffle=False, use_multiprocessing=True
            ,callbacks = [callback]
            )

 

  #Predict
  y_prediction = model3.predict(X_test)
  y_prediction  = np.argmax(y_prediction, axis=1)
  # Y_prediction = np_utils.to_categorical(y_prediction, nb_classes)

  #Create confusion matrix and normalizes it over predicted (columns)
  # result = confusion_matrix(y_test, y_prediction , normalize='pred')

  

  cm3 = confusion_matrix(y_test, y_prediction)
  cm3 = pd.DataFrame(cm3, range(10),range(10))
  # plt.figure(figsize = (10,10))
  # cm3
  # sns.heatmap(cm2, annot=True, annot_kws={"size": 12}) # font size
  # plt.show()

  # cm_using_weighted_new = cm3

  return cm3

In [16]:
cm = np.zeros([10,10])


for i in range(0,30):
  print(i)
  cm +=  isaranto_method()

cm_new = cm/30

0
Epoch 1/15
235/235 - 9s - loss: 0.3032 - categorical_accuracy: 0.9058 - val_loss: 0.2383 - val_categorical_accuracy: 0.9259 - 9s/epoch - 37ms/step
Epoch 2/15
235/235 - 7s - loss: 0.1173 - categorical_accuracy: 0.9645 - val_loss: 0.1106 - val_categorical_accuracy: 0.9666 - 7s/epoch - 28ms/step
Epoch 3/15
235/235 - 7s - loss: 0.0788 - categorical_accuracy: 0.9756 - val_loss: 0.0989 - val_categorical_accuracy: 0.9701 - 7s/epoch - 30ms/step
Epoch 4/15
235/235 - 8s - loss: 0.0593 - categorical_accuracy: 0.9825 - val_loss: 0.1007 - val_categorical_accuracy: 0.9699 - 8s/epoch - 35ms/step
Epoch 5/15
235/235 - 7s - loss: 0.0480 - categorical_accuracy: 0.9842 - val_loss: 0.0817 - val_categorical_accuracy: 0.9772 - 7s/epoch - 28ms/step
Epoch 6/15
235/235 - 8s - loss: 0.0393 - categorical_accuracy: 0.9876 - val_loss: 0.0869 - val_categorical_accuracy: 0.9751 - 8s/epoch - 33ms/step
Epoch 7/15
235/235 - 9s - loss: 0.0325 - categorical_accuracy: 0.9897 - val_loss: 0.0983 - val_categorical_accuracy:

In [20]:
print("This is the average confusion matrix using 30 runs")
round(cm_new,1)

This is the average confusion matrix using 30 runs


,0,1,2,3,4,5,6,7,8,9
0,972.3,0.9,0.5,0.5,0.4,1.1,1.6,0.5,1.3,0.9
1,0.0,1129.2,1.2,0.9,0.1,0.3,1.3,0.0,2.0,0.0
2,2.5,2.3,1012.4,2.4,1.8,0.1,1.4,3.7,5.1,0.3
3,0.1,0.0,3.6,990.1,0.1,5.5,0.0,3.7,2.2,4.7
4,0.7,0.0,3.4,0.0,966.4,0.0,3.0,0.6,0.5,7.4
5,2.0,0.0,0.0,5.1,1.2,878.1,1.2,0.3,2.3,1.8
6,4.0,2.7,0.2,0.7,5.6,16.6,926.8,0.0,1.5,0.0
7,1.4,3.2,9.4,0.7,3.7,0.1,0.0,991.4,4.0,14.0
8,3.0,1.1,3.0,5.4,4.1,7.4,0.5,2.4,940.1,7.1
9,1.0,2.2,0.0,2.3,10.9,2.3,0.3,1.3,0.5,988.1


In [26]:
print("The most common misclassifications are")
np.sort(round(cm_new,1))

The most common misclassifications are


array([[4.0000e-01, 5.0000e-01, 5.0000e-01, 5.0000e-01, 9.0000e-01,
        9.0000e-01, 1.1000e+00, 1.3000e+00, 1.6000e+00, 9.7230e+02],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, 1.0000e-01, 3.0000e-01,
        9.0000e-01, 1.2000e+00, 1.3000e+00, 2.0000e+00, 1.1292e+03],
       [1.0000e-01, 3.0000e-01, 1.4000e+00, 1.8000e+00, 2.3000e+00,
        2.4000e+00, 2.5000e+00, 3.7000e+00, 5.1000e+00, 1.0124e+03],
       [0.0000e+00, 0.0000e+00, 1.0000e-01, 1.0000e-01, 2.2000e+00,
        3.6000e+00, 3.7000e+00, 4.7000e+00, 5.5000e+00, 9.9010e+02],
       [0.0000e+00, 0.0000e+00, 0.0000e+00, 5.0000e-01, 6.0000e-01,
        7.0000e-01, 3.0000e+00, 3.4000e+00, 7.4000e+00, 9.6640e+02],
       [0.0000e+00, 0.0000e+00, 3.0000e-01, 1.2000e+00, 1.2000e+00,
        1.8000e+00, 2.0000e+00, 2.3000e+00, 5.1000e+00, 8.7810e+02],
       [0.0000e+00, 0.0000e+00, 2.0000e-01, 7.0000e-01, 1.5000e+00,
        2.7000e+00, 4.0000e+00, 5.6000e+00, 1.6600e+01, 9.2680e+02],
       [0.0000e+00, 1.0000e-01, 7.0000e-0